# swm-chatbot

Instalación dependencias

In [1]:
!pip install --quiet streamlit google-cloud-aiplatform langchain-google-vertexai google-cloud-vertexai langchain-community pypdf

Crear una regla de firewall

In [2]:
%%bash
gcloud compute firewall-rules create allow-imc-streamlit \
    --direction=INGRESS \
    --priority=1000 \
    --network=default \
    --action=ALLOW \
    --rules=tcp:8502 \
    --source-ranges=0.0.0.0/0 2> /dev/null

gcloud compute firewall-rules list --filter name='allow-imc-streamlit' 2> /dev/null

NAME                 NETWORK  DIRECTION  PRIORITY  ALLOW     DENY  DISABLED
allow-imc-streamlit  default  INGRESS    1000      tcp:8502        False
NAME                 NETWORK  DIRECTION  PRIORITY  ALLOW     DENY  DISABLED
allow-imc-streamlit  default  INGRESS    1000      tcp:8502        False


Descargar archivos desde Google Drive

In [ ]:
%%bash
sudo pip install gdown --quiet

sudo gdown --fuzzy "https://drive.google.com/file/d/10Kx0tp-xBWYpfb1kB11bwvleQxGx_K-Y/view?usp=sharing" --output data/
sudo gdown --fuzzy "https://drive.google.com/file/d/1D_K45gpMkvAT5LMPZEc0xdp7M0UAObY4/view?usp=sharing" --output data/

sudo chown jupyter:jupyter -R data/

Activación de API de búsqueda

In [ ]:
%%bash
gcloud services enable discoveryengine.googleapis.com
gcloud services list --enabled | grep -e 'discoveryengine'

Creación de la aplicación

In [3]:
%%writefile swm-chat.py
import streamlit as st
import google.auth;
import vertexai
from langchain_community.document_loaders import PyPDFLoader
import os
from vertexai.generative_models import GenerativeModel, ChatSession, GenerationConfig, Tool, Part, FunctionDeclaration

credentials, project_id = google.auth.default()
vertexai.init(project=project_id, location="us-central1")
foundation_models = ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro"]
contexto = """
Eres un asistente virtual que ayuda a nadadores a obtener información sobre el mundo de la natación,
como por ejemplo extraer datos de las normativas de las competiciones, records y marcas personales de otros nadadores o
noticias generales relacionadas. Es importante solo responder a consultas relacionadas con la natación.
"""

model_name = st.selectbox("Models:", foundation_models)

if "current_model" not in st.session_state:
    st.session_state.current_model = model_name

#hacer una para minimas de España (leer pdf) y otra para records (leer página/API)
#para leer pdfs igual interesa meterlo en drive
#se puede probar la API de youtube para buscar cosas de fastskins

def buscar_minimas(query: str) -> str:
    archivos_pdf = [f for f in os.listdir('data') if f.endswith('.pdf')]

    if archivos_pdf:
        path = os.path.join('data', archivos_pdf[0])
        loader = PyPDFLoader(path)

        # Esto carga el PDF y lo divide por páginas automáticamente
        documentos = loader.load()

        print(f"Se cargaron {len(documentos)} páginas.")
        print("Contenido de la página 1:")
        print(documentos[0].page_content)
    else:
        print("No se encontraron PDFs en la carpeta data/")


def nivel_natacion_libres(metros: int, minutos: int) -> str:
    if (metros == 50):
            if minutos > 1:
                return "nivel municipal"
            elif minutos < 1:
                return "nivel nacional"
            else:
                return "nivel regional"
    else:
        return "muchos metros"

herramientas_mapeo =  {"nivel_natacion_libres": nivel_natacion_libres,
                       "buscar_minimas": buscar_minimas
                      }

herramientas_agente = Tool(
    function_declarations =[
        FunctionDeclaration(name = "nivel_natacion_libres",
                                                parameters={
                                                    "type": "object",
                                                    "properties": {
                                                        "metros": {"type": "integer", "description": "quantity of meters swam"},
                                                        "minutos": {"type": "integer", "description": "ammount of time swam"},
                                                    },
                                                    "required": ["metros","minutos"],
                                                },
                           )
    ],
    FunctionDeclaration(name = "buscar_records",
                       )
)

model = GenerativeModel(model_name,
                        system_instruction = contexto,
                        generation_config = GenerationConfig(
                            temperature = 0.0,
                            max_output_tokens = 100
                        ),
                        tools = [herramientas_agente]
)


st.set_page_config(page_title="Aplicación Streamlit", page_icon="⚙️", layout="centered")

st.title("Chatbot")
st.subheader(f"Conectado a {project_id} con modelo: {model_name}")


if "chat" not in st.session_state and model:
    st.session_state.chat = model.start_chat()

if model:
    prompt = st.text_area("Introduce tu prompt:")
    if st.button("Generar Respuesta"):
        # --- Mostrar historial previo
        if prompt.strip():
            with st.chat_message("user"):
                st.write(prompt)
            # ----------------------------------
            #  Lógica del Agente
            # ----------------------------------
            with st.chat_message("assistant"):
                with st.spinner("El agente está pensando..."):
                    try:
                        # --- Envío del mensaje al modelo
                        response = st.session_state.chat.send_message(prompt)
                        # --- Comprobación de si el Agente necesita usar una herramienta (Function Calling)
                        part = response.candidates[0].content.parts[0]
                        if part.function_call:
                            st.info(f"🛠️ El agente está usando la herramienta: `{part.function_call.name}`")
                            # --- Extraer nombre de la función y argumentos
                            fn_name = part.function_call.name
                            fn_args = {key: val for key, val in part.function_call.args.items()}
                            # --- Ejecutar la función real de Python
                            if fn_name in herramientas_mapeo:
                                api_response = herramientas_mapeo[fn_name](**fn_args)
                                # --- Devolver el resultado de la función al modelo para que genere la respuesta final
                                response = st.session_state.chat.send_message(
                                    Part.from_function_response(
                                        name=fn_name,
                                        response={
                                            "content": api_response,
                                        },
                                    )
                                )
                        # --- Mostrar la respuesta final (texto natural)
                        st.write(response.text)
                    except Exception as e:
                        st.error(f"Error en la lógica del agente: {str(e)}")
else:
    st.error("Error de inicialización. Revisa logs.")


#prompt = st.chat_input("Say something")
#if prompt:
#    response = model.generate_content(prompt)
#    st.write(f"User has sent the following prompt: {response.text}")

Writing swm-chat.py


In [ ]:
!streamlit run swm-chat.py --server.port 8502




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://10.150.0.3:8502
  External URL: http://34.48.165.177:8502

/opt/conda/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
